In [ ]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm
from geopy.distance import geodesic
import random
import datetime
# from datetime import datetime, timedelta, strftime  # Импортируйте strftime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# adresses = pd.read_csv('london_postcodes-ons-postcodes-directory-feb22.csv', delimiter=',')
adresses = pd.read_csv('London_postcodes.csv', delimiter=',')

adresses.head()

In [ ]:
taxi_rides = pd.DataFrame(
    columns=['driver_id', 'trip_id', 'client_id', 'start_point', 'end_point', 'distance', 'start_time',
             'end_time', 'road_time', 'cost', 'driver_rate', 'client_rate', 'category_driver_feedback', 
             'text_driver_feedback', 'category_client_feedback', 'text_client_feedback'
             ]
    )

In [ ]:
num_rows = 5_000_000
num_drivers = 5000
num_clients = 2000

start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2024, 1, 1)


driver_id_range = range(1, num_drivers + 1)
client_id_range = range(1, num_clients + 1)


random_addresses = adresses.sample(n=num_rows, random_state=42, replace=True)

# Заполняем столбцы start_point и end_point случайными парами широты и долготы
taxi_rides['start_point'] = list(zip(random_addresses['Latitude'], random_addresses['Longitude']))
taxi_rides['end_point'] = list(zip(random_addresses['Latitude'].sample(frac=1, random_state=42), random_addresses['Longitude'].sample(frac=1, random_state=42)))

taxi_rides['driver_id'] = [random.choice(driver_id_range) for i in range(taxi_rides.shape[0])]
taxi_rides['client_id'] = [random.choice(client_id_range) for i in range(taxi_rides.shape[0])]

# Рассчитываем расстояние
taxi_rides['distance'] = [geodesic(start, end).km for start, end in taxi_rides[['start_point', 'end_point']].values]
taxi_rides['distance'] = taxi_rides['distance'].round(2)

taxi_rides['start_time'] = [datetime.datetime.strftime(start_date + datetime.timedelta(seconds=random.randint(0, 31536000)), '%Y-%m-%d %H:%M:%S') for i in range(num_rows)]
taxi_rides['road_time'] = [datetime.timedelta(seconds=distance / 50 * 3600) for distance in taxi_rides['distance']]
taxi_rides['end_time'] = [datetime.datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S') + road_time for start_time, road_time in zip(taxi_rides['start_time'], taxi_rides['road_time'])]

taxi_rides['start_time'] = pd.to_datetime(taxi_rides['start_time'], format='%Y-%m-%d %H:%M:%S')
taxi_rides['end_time'] = pd.to_datetime(taxi_rides['end_time'], format='%Y-%m-%d %H:%M:%S')

taxi_rides['trip_id'] = range(1, len(taxi_rides) + 1)


In [ ]:
def count_cost(start_time, distance):
    cost = 5 + .5 * distance
    if (start_time.hour >= 8 and start_time.hour <= 11) or (start_time.hour >= 17 and start_time.hour <= 19):
        cost *= 1.5

    if (start_time.hour >= 23 and start_time.hour <= 6):
        cost *= 1.75
    return cost


taxi_rides['cost'] = [count_cost(time, dist) for time, dist in zip(taxi_rides['start_time'], taxi_rides['distance'])]
taxi_rides['cost'] = taxi_rides['cost'].round(2)

In [ ]:
driver_rate_idx = np.random.randint(low=0, high=num_drivers, size=int(num_rows*0.85))
driver_rate_distr_arr = np.random.multinomial(1, [.2, .05, .1, .25, .4], size=int(num_rows*0.85))
taxi_rides['driver_rate'][driver_rate_idx] = np.where(driver_rate_distr_arr == 1)[1] + 1

client_rate_idx = np.random.randint(low=0, high=num_drivers, size=int(num_rows*0.75))
client_rate_distr_arr = np.random.multinomial(1, [.2, .05, .1, .25, .4], size=int(num_rows*0.75))
taxi_rides['client_rate'][driver_rate_idx] = np.where(driver_rate_distr_arr == 1)[1] + 1

In [ ]:
driver_fb_cat_good = ['good service', 'nice car', 'expert navigation']
driver_fb_cat_bad = ['awful service', 'bad car', 'dirty', 'rude']

client_fb_cat_good = ['polite', 'pleasant', 'quiet']
client_fb_cat_bad = ['unpolite', 'unpleasant', 'loud', 'dirty']

category_driver_good_feedback_idx = np.random.choice(taxi_rides[taxi_rides.driver_rate > 3].index, size=int(num_rows*0.3*0.2))
taxi_rides["category_driver_feedback"][category_driver_good_feedback_idx] = np.random.choice(driver_fb_cat_good, size=int(num_rows*0.3*0.2))

category_driver_bad_feedback_idx = np.random.choice(taxi_rides[taxi_rides.driver_rate < 4].index, size=int(num_rows*0.3*0.2))
taxi_rides["category_driver_feedback"][category_driver_bad_feedback_idx] = np.random.choice(driver_fb_cat_bad, size=int(num_rows*0.3*0.2))


text_good_feedback_driver_length = np.random.randint(low=0, high=len(driver_fb_cat_good), size=int(num_rows*0.3*0.2))
text_good_feedback_driver_sample = [random.sample(driver_fb_cat_good, i) for i in text_good_feedback_driver_length]
taxi_rides['text_driver_feedback'][category_driver_good_feedback_idx] = text_good_feedback_driver_sample

text_bad_feedback_driver_length = np.random.randint(low=0, high=len(driver_fb_cat_bad), size=int(num_rows*0.3*0.2))
text_bad_feedback_driver_sample = [random.sample(driver_fb_cat_bad, i) for i in text_bad_feedback_driver_length]
taxi_rides['text_driver_feedback'][category_driver_bad_feedback_idx] = text_bad_feedback_driver_sample

# ------------------

category_client_good_feedback_idx = np.random.choice(taxi_rides[taxi_rides.client_rate > 3].index, size=int(num_rows*0.3*0.2))
taxi_rides["category_client_feedback"][category_client_good_feedback_idx] = np.random.choice(client_fb_cat_good, size=int(num_rows*0.3*0.2))

category_client_bad_feedback_idx = np.random.choice(taxi_rides[taxi_rides.client_rate < 4].index, size=int(num_rows*0.3*0.2))
taxi_rides["category_client_feedback"][category_client_bad_feedback_idx] = np.random.choice(client_fb_cat_bad, size=int(num_rows*0.3*0.2))


text_good_feedback_client_length = np.random.randint(low=0, high=len(client_fb_cat_good), size=int(num_rows*0.3*0.2))
text_good_feedback_client_sample = [random.sample(client_fb_cat_good, i) for i in text_good_feedback_client_length]
taxi_rides['text_client_feedback'][category_client_good_feedback_idx] = text_good_feedback_client_sample

text_bad_feedback_client_length = np.random.randint(low=0, high=len(client_fb_cat_bad), size=int(num_rows*0.3*0.2))
text_bad_feedback_client_sample = [random.sample(client_fb_cat_bad, i) for i in text_bad_feedback_client_length]
taxi_rides['text_client_feedback'][category_client_bad_feedback_idx] = text_bad_feedback_client_sample


In [ ]:
taxi_rides

In [ ]:
taxi_rides.to_csv('taxi.csv',  index=False)